In [1]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리.csv')

In [2]:
df = data_raw[['만나이','성별','우울감 경험 여부','자치구']]

In [3]:
df_dep1=df[df['우울감 경험 여부'] != 9]
df_dep2=df_dep1[df_dep1['우울감 경험 여부'] != 7]
df_dep2

,만나이,성별,우울감 경험 여부,자치구
0,61,2,2,마포구
1,66,1,2,마포구
2,89,2,2,마포구
3,27,1,2,마포구
4,42,2,2,마포구
...,...,...,...,...
22925,62,2,2,금천구
22926,65,1,2,금천구
22927,24,1,2,금천구
22928,56,2,2,금천구


In [4]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [5]:
list_df = []
dic_all ={}
for value in list_region:
    temp = df_dep2['자치구'] == value
    list_df.append(df_dep2[temp])
list_df[0] = df_dep2
list_df

[       만나이  성별  우울감 경험 여부  자치구
 0       61   2          2  마포구
 1       66   1          2  마포구
 2       89   2          2  마포구
 3       27   1          2  마포구
 4       42   2          2  마포구
 ...    ...  ..        ...  ...
 22925   62   2          2  금천구
 22926   65   1          2  금천구
 22927   24   1          2  금천구
 22928   56   2          2  금천구
 22929   66   2          2  금천구
 
 [22909 rows x 4 columns],
       만나이  성별  우울감 경험 여부   자치구
 6419   64   2          2  영등포구
 6420   23   1          2  영등포구
 6421   49   2          2  영등포구
 6422   51   1          2  영등포구
 6423   54   1          2  영등포구
 ...   ...  ..        ...   ...
 7330   25   2          2  영등포구
 7331   20   2          2  영등포구
 7332   51   2          2  영등포구
 7333   54   1          2  영등포구
 7334   26   1          2  영등포구
 
 [914 rows x 4 columns],
        만나이  성별  우울감 경험 여부  자치구
 18346   64   2          2  양천구
 18347   68   1          2  양천구
 18348   66   1          2  양천구
 18349   62   2          2  양천구
 18350   66   1 

In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_dep = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] < int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            value_dep.append(round(100 - temp2['우울감 경험 여부'].value_counts(normalize = True)[2]*100,1))
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [20]:
df1 = pd.DataFrame({'우울감 경험률':value_dep, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df1 = df1.set_index(['지역','연령별','성별'])

df1

우울감 경험률
지역   연령별   성별         
서울시  계     계       6.3
           남       4.6
           여       7.7
     19~44 계       6.5
           남       4.9
           여       7.8
     45~64 계       6.0
           남       4.6
           여       7.0
     65~74 계       6.2
           남       4.3
           여       7.6
     75 이상 계       7.4
           남       4.6
           여       9.4
영등포구 계     계       8.1
           남       4.2
           여      11.5
     19~44 계       9.0
           남       3.6
           여      14.0
     45~64 계       7.1
           남       5.0
           여       8.8
     65~74 계       9.7
           남       4.9
           여      13.3
     75 이상 계       2.7
           남       2.5
           여       2.9
양천구  계     계       6.9
           남       6.1
           여       7.5
     19~44 계       6.6
           남       4.9
           여       7.9
     45~64 계       7.2
           남       7.5
           여       7.0
     65~74 계       6.5
           남       4.2
           여       8.3
     75 이상 계       7.0
           남       8.0
           여       6.5
마포구  계     계       7.7
           남       4.6
           여      10.1
     19~44 계       8.5
           남       5.9
           여      10.5
     45~64 계       7.4
           남       3.6
           여      10.1
     65~74 계       9.7
           남       4.2
           여      13.8
     75 이상 계       4.8
           남       4.3
           여       5.3
동작구  계     계       4.9
           남       2.5
           여       6.9
     19~44 계       2.8
           남       1.2
           여       4.2
     45~64 계       5.1
           남       3.4
           여       6.4
     65~74 계       3.6
           남       3.1
           여       4.0
     75 이상 계      14.4
           남       5.4
           여      20.0
금천구  계     계       7.9
           남       5.9
           여       9.5
     19~44 계       4.7
           남       4.5
           여       4.8
     45~64 계      10.0
           남       7.6
           여      11.9
     65~74 계       8.5
           남       3.3
           여      14.0
     75 이상 계      11.4
           남       7.3
           여      14.9
구로구  계     계       5.6
           남       2.7
           여       7.8
     19~44 계       4.1
           남       2.3
           여       5.6
     45~64 계       4.9
           남       2.5
           여       6.8
     65~74 계       8.0
           남       6.2
           여       9.1
     75 이상 계       7.4
           남       2.2
           여      12.5
관악구  계     계       6.6
           남       5.3
           여       7.5
     19~44 계       6.0
           남       5.7
           여       6.2
     45~64 계       9.3
           남       8.1
           여      10.0
     65~74 계       4.4
           남       2.2
           여       5.6
     75 이상 계       5.3
           남       2.1
           여       8.3
강서구  계     계       5.7
           남       3.2
           여       7.6
     19~44 계       6.0
           남       1.9
           여       9.6
     45~64 계       5.4
           남       5.3
           여       5.5
     65~74 계       4.4
           남       2.1
           여       6.2
     75 이상 계       4.8
           남       5.3
           여       4.3
중랑구  계     계       7.4
           남       4.8
           여       9.2
     19~44 계       8.8
           남       7.4
           여       9.8
     45~64 계       7.4
           남       3.4
           여       9.5
     65~74 계       5.3
           남       4.0
           여       6.7
     75 이상 계       8.1
           남       5.6
           여      10.0
종로구  계     계       3.2
           남       2.7
           여       3.5
     19~44 계       5.0
           남       5.8
           여       4.2
     45~64 계       1.9
           남       0.6
           여       2.9
     65~74 계       0.9
           남       2.1
           여       0.0
     75 이상 계       5.7
           남       2.3
           여       7.6
은평구  계     계       9.8
           남       6.7
           여      11.9
     19~44 계       8.5
           남       6.2
           여      10.1
     45~64 계       6.4
           남 